In [ ]:
import numpy as np
import pandas as pd
import configparser
import tensorflow as tf
from sklearn import model_selection, utils

In [ ]:
from ipynb.fs.full.ImportExportData import exportDataframeToCSV,importCsvToDataframe
from ipynb.fs.full.EdfManipulation import read_and_store_data
from ipynb.fs.full.ClassificationPerformanceIndexes import classificationPerformanceIndexes
from ipynb.fs.full.FeatureClassificationMethods import *
from ipynb.fs.full.FeatureComputation import averageChannels, featureExtractionAverage
from ipynb.fs.full.FeatureNormalization import featureNormalization
from ipynb.fs.full.FeatureSelection import dimentionalityReduction
from ipynb.fs.full.DatasetBalancing import oversamplingSMOTE
from ipynb.fs.full.SplitDataset import createTrainingAndTestDatasets

### Feature Extraction

In [ ]:
def AverageChannelFeatures(df, readEdf, sample_rate, step, pca_tolerance, smote_neighbors, csvExportFile):
    if readEdf:
        ndf = averageChannels(df)
        features = pd.DataFrame(featureExtractionAverage (ndf, sample_rate, step))
        del ndf
        features = featureNormalization(features)
        reduced_features = dimentionalityReduction(features, pca_tolerance)
        del features
        smote_features = utils.shuffle(oversamplingSMOTE(reduced_features.drop('seizure', axis = 1), reduced_features['seizure'], smote_neighbors))
        smote_features.reset_index(drop = True, inplace = True)
        del reduced_features
        exportDataframeToCSV(smote_features, csvExportFile)
    else :
        smote_features = df
    return smote_features

### Create Training and Testing Data

In [ ]:
def AverageChannelTrainTestData (smote_features, test_ratio, k_fold, perfInd):
    train_dat, test_dat, train_ind, test_ind = createTrainingAndTestDatasets(smote_features, test_ratio)
    results = pd.DataFrame(columns = perfInd)
    kf = model_selection.KFold(n_splits = k_fold, shuffle=True)
    return train_dat, test_dat, train_ind, test_ind, results, kf

### Experiment

In [ ]:
def AverageChannelExperiment(df, channels, readEdf, csvImportFile, csvExportFile, dataset, sample_rate, time_window, test_ratio, pca_tolerance, smote_neighbors, k_fold, knn_neighbors, rf_estimators, max_iter, dropout, loss_function, epochs, lstm_units, dense1_units, dense2_units, batch, step):
    perfInd = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1 Score', 'MCC', 'Kappa']
    smote_features = AverageChannelFeatures(df, readEdf, sample_rate, step, pca_tolerance, smote_neighbors, csvExportFile)
    train_dat, test_dat, train_ind, test_ind, results, kf = AverageChannelTrainTestData (smote_features, test_ratio, k_fold, perfInd)
    CompleteSVM(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd)
    CompleteKNN(train_dat, test_dat, train_ind, test_ind, results, knn_neighbors, smote_features, kf, perfInd)
    CompleteNB(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd)
    CompleteDT(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd)
    CompleteRF(train_dat, test_dat, train_ind, test_ind, results, rf_estimators, smote_features, kf, perfInd)
    CompleteLDA(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd)
    CompleteLR(train_dat, test_dat, train_ind, test_ind, results, max_iter, smote_features, kf, perfInd)
    CompleteLSTM(train_dat, test_dat, train_ind, test_ind, results, smote_features, kf, perfInd, lstm_units, dense1_units, dense2_units, dropout, epochs, loss_function, batch)
    return results